# Recreating Pixel Maps with static pixels in percentile bins

In [1]:
%matplotlib inline

from matplotlib.pyplot import subplots, savefig
import richardsplot as rplot

from utilities.analysis import plotting
from utilities.ccd import CCD
from utilities.containers import *
from utilities.conversions import magToFlux

In [2]:
cmap = rplot.palettable.colorbrewer.diverging.PuOr_11.mpl_colormap

In [3]:
!pwd
!mkdir plots/c08/

/home/jackeline/Research/k2_pipeline/CARMA
mkdir: cannot create directory ‘plots/c08/’: File exists


# Load Data hdf5 file --example using C8 M10.1

In [4]:
pixmapfile = "/media/jackeline/UUI/PixelMaps/K2_target_pixels.hdf5"

#  Run the Pixel Map Generator

In [5]:
ccd = CCD(module=10, channel=1, field='FLUX', campaign=8)
cont = PixelMapContainer.from_hdf5(pixmapfile, ccd)
cont.save(pixmapfile)
gen = PixMapGenerator(cont)
print "Making", ccd
            

  Writing... (Do Not Turn off Device or Stop Kernel)

Making CCD(module=10, channel=1, field='FLUX', campaign=8)


# tracking pixels

In [37]:
from itertools import izip, imap, ifilter
import numpy as np

def test_pixel_lc(gen,percentiles, flux_range=None, stat_funcs=None):
    if flux_range is not None:
        lower, higher = sorted(flux_range)

    if stat_funcs is None:
        stat_funcs = (np.median,)

    ccd = gen.ccd
    N = gen.N
    M = len(percentiles)-1
    L = len(stat_funcs)

    lc = np.empty((N, M, L+2))
    lc[::] = np.nan


    l_bin_edges = np.empty((gen.N, len(percentiles)))
    def _get_epochs():

        if flux_range is not None:
            return _get_filtered_epochs()

        return _get_full_epochs()

    def _get_full_epochs():

        for i, g in enumerate(imap(gen.get_unordered, xrange(gen.N))):

                yield g

    def _get_filtered_epochs():

        for i, g in enumerate(imap(gen.get_unordered, xrange(gen.N))):

            mask = (g > lower) & (g < higher)
            yield g[mask]
    
    pixel_median_flux = []

    for i in range(len(gen.containers.containers[:])):
        pix_t_index = np.shape(gen.containers.containers[i].pixels)[0]
        pix_row = np.shape(gen.containers.containers[i].pixels)[1]
        pix_column = np.shape(gen.containers.containers[i].pixels)[2]

        for r in range (pix_row):
            for c in range (pix_column):
                fluxes = gen.containers.containers[i].pixels[:,r,c]
                mask = (fluxes > lower) & (fluxes < higher)
                pixel_median_flux.append([r,c,np.nanmedian(fluxes[mask])])
    print("computed median fluxes for %d pixels" %len(pixel_median_flux))    
    #remove neg flux pixels and nans
    keep = np.where((np.array(pixel_median_flux)[:,2])>0)[0] 
    print('Removing %d pixels due to negative flux or NANs' %(len(pixel_median_flux)-len(keep)))
    #assign pixel a percentile based on median
    percentile_bin_edges = np.nanpercentile(np.array(pixel_median_flux)[keep,2], percentiles, axis=0).T
    percentile_bin_edges[-1]  += 1e-10 
    indices = np.digitize(np.array(pixel_median_flux)[keep,2], percentile_bin_edges)-1 
    print('%d indices are negative and %d indices are greater than 499 '%(len(indices[indices < 0 ]), len(indices[indices > 499 ])))
    

    #lengths = np.array(map(len, epochs))
    #unique_lengths = np.unique(lengths[lengths!=0])
    

    #lc[:,:,0] = l_bin_edges[:-1]
    #lc[:,:,1] = l_bin_edges[1:]

    #outer loop over each epoch  (time step)  
    for l in range(pix_t_index):
        allpixel = []
        #first inner loop over each postage stamp
        for i in range(len(gen.containers.containers[:])):
            pix_t_index = np.shape(gen.containers.containers[i].pixels)[0]
            pix_row = np.shape(gen.containers.containers[i].pixels)[1]
            pix_column = np.shape(gen.containers.containers[i].pixels)[2]
            #2nd inner loop over each pixel coordinate (r,c)
            for r in range (pix_row):
                for c in range (pix_column):
                    #stack all pixels all at epoch = l index 
                    allpixel.append(gen.containers.containers[i].pixels[l,r,c])
        # y is a pixel stack for the current epoch, y is updated for each iteration over the outer loop-l            
        y = np.array(allpixel)[keep]
        
        #*********test****** keep lines below commented out or pixels will move across bins
        #indices = np.digitize(y, percentile_bin_edges)-1 # digitize offsets the index
        #indices[indices < 0] = 0
        #indices[indices >=500] = 500-1
        #*******************
        
        #count number of pixel in each bin per each epoch pixels with target centroids are removed
        print(l,len(y), len(indices))
        
        #calculated the number of non-zero percentile bins
        bin_counts = np.bincount(indices) # try to arrange arrays so that we can compute stats in parallel
        same_counts = np.unique(bin_counts[bin_counts!=0])

        #compute value of each percentile bin for current epoch
        for count in same_counts:

            index = np.where(bin_counts == count)[0]
            uniform_array = np.asarray([y[i==indices] for i in index])

            lc[l,index, 2:] = np.array([stat_func(uniform_array, axis=1) for stat_func in stat_funcs]).T
    return lc
        

In [38]:
percentiles = np.linspace(0, 100, 501)
flux_range = map(magToFlux,(50,1))
lc = test_pixel_lc(gen,percentiles, flux_range = flux_range)


/home/jackeline/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in greater
/home/jackeline/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in less


computed median fluxes for 23530 pixels
Removing 516 pixels due to negative flux or NANs
0 indices are negative and 0 indices are greater than 499 
(0, 23014, 23014)
(1, 23014, 23014)
(2, 23014, 23014)
(3, 23014, 23014)
(4, 23014, 23014)
(5, 23014, 23014)
(6, 23014, 23014)
(7, 23014, 23014)
(8, 23014, 23014)
(9, 23014, 23014)
(10, 23014, 23014)
(11, 23014, 23014)
(12, 23014, 23014)
(13, 23014, 23014)
(14, 23014, 23014)
(15, 23014, 23014)
(16, 23014, 23014)
(17, 23014, 23014)
(18, 23014, 23014)
(19, 23014, 23014)
(20, 23014, 23014)
(21, 23014, 23014)
(22, 23014, 23014)
(23, 23014, 23014)
(24, 23014, 23014)
(25, 23014, 23014)
(26, 23014, 23014)
(27, 23014, 23014)
(28, 23014, 23014)
(29, 23014, 23014)
(30, 23014, 23014)
(31, 23014, 23014)
(32, 23014, 23014)
(33, 23014, 23014)
(34, 23014, 23014)
(35, 23014, 23014)
(36, 23014, 23014)
(37, 23014, 23014)
(38, 23014, 23014)
(39, 23014, 23014)
(40, 23014, 23014)
(41, 23014, 23014)
(42, 23014, 23014)
(43, 23014, 23014)
(44, 23014, 23014)
(45, 23

(417, 23014, 23014)
(418, 23014, 23014)
(419, 23014, 23014)
(420, 23014, 23014)
(421, 23014, 23014)
(422, 23014, 23014)
(423, 23014, 23014)
(424, 23014, 23014)
(425, 23014, 23014)
(426, 23014, 23014)
(427, 23014, 23014)
(428, 23014, 23014)
(429, 23014, 23014)
(430, 23014, 23014)
(431, 23014, 23014)
(432, 23014, 23014)
(433, 23014, 23014)
(434, 23014, 23014)
(435, 23014, 23014)
(436, 23014, 23014)
(437, 23014, 23014)
(438, 23014, 23014)
(439, 23014, 23014)
(440, 23014, 23014)
(441, 23014, 23014)
(442, 23014, 23014)
(443, 23014, 23014)
(444, 23014, 23014)
(445, 23014, 23014)
(446, 23014, 23014)
(447, 23014, 23014)
(448, 23014, 23014)
(449, 23014, 23014)
(450, 23014, 23014)
(451, 23014, 23014)
(452, 23014, 23014)
(453, 23014, 23014)
(454, 23014, 23014)
(455, 23014, 23014)
(456, 23014, 23014)
(457, 23014, 23014)
(458, 23014, 23014)
(459, 23014, 23014)
(460, 23014, 23014)
(461, 23014, 23014)
(462, 23014, 23014)
(463, 23014, 23014)
(464, 23014, 23014)
(465, 23014, 23014)
(466, 23014, 23014)


(832, 23014, 23014)
(833, 23014, 23014)
(834, 23014, 23014)
(835, 23014, 23014)
(836, 23014, 23014)
(837, 23014, 23014)
(838, 23014, 23014)
(839, 23014, 23014)
(840, 23014, 23014)
(841, 23014, 23014)
(842, 23014, 23014)
(843, 23014, 23014)
(844, 23014, 23014)
(845, 23014, 23014)
(846, 23014, 23014)
(847, 23014, 23014)
(848, 23014, 23014)
(849, 23014, 23014)
(850, 23014, 23014)
(851, 23014, 23014)
(852, 23014, 23014)
(853, 23014, 23014)
(854, 23014, 23014)
(855, 23014, 23014)
(856, 23014, 23014)
(857, 23014, 23014)
(858, 23014, 23014)
(859, 23014, 23014)
(860, 23014, 23014)
(861, 23014, 23014)
(862, 23014, 23014)
(863, 23014, 23014)
(864, 23014, 23014)
(865, 23014, 23014)
(866, 23014, 23014)
(867, 23014, 23014)
(868, 23014, 23014)
(869, 23014, 23014)
(870, 23014, 23014)
(871, 23014, 23014)
(872, 23014, 23014)
(873, 23014, 23014)
(874, 23014, 23014)
(875, 23014, 23014)
(876, 23014, 23014)
(877, 23014, 23014)
(878, 23014, 23014)
(879, 23014, 23014)
(880, 23014, 23014)
(881, 23014, 23014)


(1236, 23014, 23014)
(1237, 23014, 23014)
(1238, 23014, 23014)
(1239, 23014, 23014)
(1240, 23014, 23014)
(1241, 23014, 23014)
(1242, 23014, 23014)
(1243, 23014, 23014)
(1244, 23014, 23014)
(1245, 23014, 23014)
(1246, 23014, 23014)
(1247, 23014, 23014)
(1248, 23014, 23014)
(1249, 23014, 23014)
(1250, 23014, 23014)
(1251, 23014, 23014)
(1252, 23014, 23014)
(1253, 23014, 23014)
(1254, 23014, 23014)
(1255, 23014, 23014)
(1256, 23014, 23014)
(1257, 23014, 23014)
(1258, 23014, 23014)
(1259, 23014, 23014)
(1260, 23014, 23014)
(1261, 23014, 23014)
(1262, 23014, 23014)
(1263, 23014, 23014)
(1264, 23014, 23014)
(1265, 23014, 23014)
(1266, 23014, 23014)
(1267, 23014, 23014)
(1268, 23014, 23014)
(1269, 23014, 23014)
(1270, 23014, 23014)
(1271, 23014, 23014)
(1272, 23014, 23014)
(1273, 23014, 23014)
(1274, 23014, 23014)
(1275, 23014, 23014)
(1276, 23014, 23014)
(1277, 23014, 23014)
(1278, 23014, 23014)
(1279, 23014, 23014)
(1280, 23014, 23014)
(1281, 23014, 23014)
(1282, 23014, 23014)
(1283, 23014,

...

(2034, 23014, 23014)
(2035, 23014, 23014)
(2036, 23014, 23014)
(2037, 23014, 23014)
(2038, 23014, 23014)
(2039, 23014, 23014)
(2040, 23014, 23014)
(2041, 23014, 23014)
(2042, 23014, 23014)
(2043, 23014, 23014)
(2044, 23014, 23014)
(2045, 23014, 23014)
(2046, 23014, 23014)
(2047, 23014, 23014)
(2048, 23014, 23014)
(2049, 23014, 23014)
(2050, 23014, 23014)
(2051, 23014, 23014)
(2052, 23014, 23014)
(2053, 23014, 23014)
(2054, 23014, 23014)
(2055, 23014, 23014)
(2056, 23014, 23014)
(2057, 23014, 23014)
(2058, 23014, 23014)
(2059, 23014, 23014)
(2060, 23014, 23014)
(2061, 23014, 23014)
(2062, 23014, 23014)
(2063, 23014, 23014)
(2064, 23014, 23014)
(2065, 23014, 23014)
(2066, 23014, 23014)
(2067, 23014, 23014)
(2068, 23014, 23014)
(2069, 23014, 23014)
(2070, 23014, 23014)
(2071, 23014, 23014)
(2072, 23014, 23014)
(2073, 23014, 23014)
(2074, 23014, 23014)
(2075, 23014, 23014)
(2076, 23014, 23014)
(2077, 23014, 23014)
(2078, 23014, 23014)
(2079, 23014, 23014)
(2080, 23014, 23014)
(2081, 23014,

(2430, 23014, 23014)
(2431, 23014, 23014)
(2432, 23014, 23014)
(2433, 23014, 23014)
(2434, 23014, 23014)
(2435, 23014, 23014)
(2436, 23014, 23014)
(2437, 23014, 23014)
(2438, 23014, 23014)
(2439, 23014, 23014)
(2440, 23014, 23014)
(2441, 23014, 23014)
(2442, 23014, 23014)
(2443, 23014, 23014)
(2444, 23014, 23014)
(2445, 23014, 23014)
(2446, 23014, 23014)
(2447, 23014, 23014)
(2448, 23014, 23014)
(2449, 23014, 23014)
(2450, 23014, 23014)
(2451, 23014, 23014)
(2452, 23014, 23014)
(2453, 23014, 23014)
(2454, 23014, 23014)
(2455, 23014, 23014)
(2456, 23014, 23014)
(2457, 23014, 23014)
(2458, 23014, 23014)
(2459, 23014, 23014)
(2460, 23014, 23014)
(2461, 23014, 23014)
(2462, 23014, 23014)
(2463, 23014, 23014)
(2464, 23014, 23014)
(2465, 23014, 23014)
(2466, 23014, 23014)
(2467, 23014, 23014)
(2468, 23014, 23014)
(2469, 23014, 23014)
(2470, 23014, 23014)
(2471, 23014, 23014)
(2472, 23014, 23014)
(2473, 23014, 23014)
(2474, 23014, 23014)
(2475, 23014, 23014)
(2476, 23014, 23014)
(2477, 23014,

(2826, 23014, 23014)
(2827, 23014, 23014)
(2828, 23014, 23014)
(2829, 23014, 23014)
(2830, 23014, 23014)
(2831, 23014, 23014)
(2832, 23014, 23014)
(2833, 23014, 23014)
(2834, 23014, 23014)
(2835, 23014, 23014)
(2836, 23014, 23014)
(2837, 23014, 23014)
(2838, 23014, 23014)
(2839, 23014, 23014)
(2840, 23014, 23014)
(2841, 23014, 23014)
(2842, 23014, 23014)
(2843, 23014, 23014)
(2844, 23014, 23014)
(2845, 23014, 23014)
(2846, 23014, 23014)
(2847, 23014, 23014)
(2848, 23014, 23014)
(2849, 23014, 23014)
(2850, 23014, 23014)
(2851, 23014, 23014)
(2852, 23014, 23014)
(2853, 23014, 23014)
(2854, 23014, 23014)
(2855, 23014, 23014)
(2856, 23014, 23014)
(2857, 23014, 23014)
(2858, 23014, 23014)
(2859, 23014, 23014)
(2860, 23014, 23014)
(2861, 23014, 23014)
(2862, 23014, 23014)
(2863, 23014, 23014)
(2864, 23014, 23014)
(2865, 23014, 23014)
(2866, 23014, 23014)
(2867, 23014, 23014)
(2868, 23014, 23014)
(2869, 23014, 23014)
(2870, 23014, 23014)
(2871, 23014, 23014)
(2872, 23014, 23014)
(2873, 23014,

(3222, 23014, 23014)
(3223, 23014, 23014)
(3224, 23014, 23014)
(3225, 23014, 23014)
(3226, 23014, 23014)
(3227, 23014, 23014)
(3228, 23014, 23014)
(3229, 23014, 23014)
(3230, 23014, 23014)
(3231, 23014, 23014)
(3232, 23014, 23014)
(3233, 23014, 23014)
(3234, 23014, 23014)
(3235, 23014, 23014)
(3236, 23014, 23014)
(3237, 23014, 23014)
(3238, 23014, 23014)
(3239, 23014, 23014)
(3240, 23014, 23014)
(3241, 23014, 23014)
(3242, 23014, 23014)
(3243, 23014, 23014)
(3244, 23014, 23014)
(3245, 23014, 23014)
(3246, 23014, 23014)
(3247, 23014, 23014)
(3248, 23014, 23014)
(3249, 23014, 23014)
(3250, 23014, 23014)
(3251, 23014, 23014)
(3252, 23014, 23014)
(3253, 23014, 23014)
(3254, 23014, 23014)
(3255, 23014, 23014)
(3256, 23014, 23014)
(3257, 23014, 23014)
(3258, 23014, 23014)
(3259, 23014, 23014)
(3260, 23014, 23014)
(3261, 23014, 23014)
(3262, 23014, 23014)
(3263, 23014, 23014)
(3264, 23014, 23014)
(3265, 23014, 23014)
(3266, 23014, 23014)
(3267, 23014, 23014)
(3268, 23014, 23014)
(3269, 23014,

(3615, 23014, 23014)
(3616, 23014, 23014)
(3617, 23014, 23014)
(3618, 23014, 23014)
(3619, 23014, 23014)
(3620, 23014, 23014)
(3621, 23014, 23014)
(3622, 23014, 23014)
(3623, 23014, 23014)
(3624, 23014, 23014)
(3625, 23014, 23014)
(3626, 23014, 23014)
(3627, 23014, 23014)
(3628, 23014, 23014)
(3629, 23014, 23014)
(3630, 23014, 23014)
(3631, 23014, 23014)
(3632, 23014, 23014)
(3633, 23014, 23014)
(3634, 23014, 23014)
(3635, 23014, 23014)
(3636, 23014, 23014)
(3637, 23014, 23014)
(3638, 23014, 23014)
(3639, 23014, 23014)
(3640, 23014, 23014)
(3641, 23014, 23014)
(3642, 23014, 23014)
(3643, 23014, 23014)
(3644, 23014, 23014)
(3645, 23014, 23014)
(3646, 23014, 23014)
(3647, 23014, 23014)
(3648, 23014, 23014)
(3649, 23014, 23014)
(3650, 23014, 23014)
(3651, 23014, 23014)
(3652, 23014, 23014)
(3653, 23014, 23014)
(3654, 23014, 23014)
(3655, 23014, 23014)
(3656, 23014, 23014)
(3657, 23014, 23014)
(3658, 23014, 23014)
(3659, 23014, 23014)
(3660, 23014, 23014)
(3661, 23014, 23014)
(3662, 23014,

# Plot CCD Pixel Map

In [ ]:
fig, ax = subplots(1,1, figsize=(16,9))
im = plotting.plot_pixel_lc(ax, lc[:,:,2], smooth=120, image=True,
                          aspect='auto', cmap=cmap, vmin=-1, vmax=1)
plotting.format_pixel_image(ax, im, lc.shape[0])
ax.set_title("Median Pixel Light Curves M{}.{}".format(ccd.module, ccd.channel),
                fontsize=30)
ax.plot(0, 0, "-", label="M{}.{}".format(ccd.module, ccd.channel+1))    
ax.legend(fontsize = 50, frameon=False, loc = 8, handlelength=0, handletextpad=0,)
savefig('plots/c{:02d}/Module{}Channel{}PixelMapFull.png'.format(ccd.campaign, ccd.module, ccd.channel))
    
#Make the differenced image map
fig, ax = subplots(1,1, figsize=(16,9))
im = plotting.plot_pixel_lc(ax, lc[:,:,2], smooth=120, image=True,
                          differenced=True, aspect='auto', cmap=cmap, vmin=-1, vmax=1)
plotting.format_pixel_image(ax, im, lc.shape[0])
ax.set_title("Differenced Median Pixel Light Curves M{}.{}".format(ccd.module, ccd.channel),
                fontsize=30)
ax.plot(0, 0, "-", label="M{}.{}".format(ccd.module, ccd.channel+1))    
ax.legend(fontsize = 50, frameon=False, loc = 8, handlelength=0, handletextpad=0,)
savefig('plots/c{:02d}/Module{}Channel{}PixelMapDiff.png'.format(ccd.campaign, ccd.module, ccd.channel))
    

In [32]:
    
    pixel_median_flux = []

    for i in range(len(gen.containers.containers[:])):
        pix_t_index = np.shape(gen.containers.containers[i].pixels)[0]
        pix_row = np.shape(gen.containers.containers[i].pixels)[1]
        pix_column = np.shape(gen.containers.containers[i].pixels)[2]

        for r in range (pix_row):
            for c in range (pix_column):
                fluxes = gen.containers.containers[i].pixels[:,r,c]
                #mask = (fluxes > lower) & (fluxes < higher)
                pixel_median_flux.append([r,c,np.nanmedian(fluxes)])
    print("computed median fluxes for %d pixels" %len(pixel_median_flux))   
    #remove neg flux pixels and nans
    keep = np.where((np.array(pixel_median_flux)[:,2])>0)[0] 
    #assign pixel a percentile based on median
    percentile_bin_edges = np.nanpercentile(np.array(pixel_median_flux)[keep,2], percentiles, axis=0).T
    percentile_bin_edges[-1]  += 1e-10 
    indices = np.digitize(np.array(pixel_median_flux)[keep,2], percentile_bin_edges)-1 
    print('%d indices are negative and %d indices are greater than 499 '%(len(indices[indices < 0 ]), len(indices[indices > 499 ])))
    

computed median fluxes for 23530 pixels
0 indices are negative and 0 indices are greater than 499 


In [31]:
keep = np.where((np.array(pixel_median_flux)[:,2])>0)[0] 